In [ ]:
"""
STEPS
1. Find the image collection corresponding to LANDSAT7 data
2. Filter the collection to January 2005 outside of the rainy season
3. Load in village location data
4. Create function to load in lat and long data from each village row
5. Function creates EE.Geometry.Point object from each lat and long and sets a buffer of a certain radius
6. Function loads in all bands of data from that image and inputs it into correspond row of new dataframe
7. Function adds village category to each row
8. Train neural network to predict whether there is a village, destroyed village, or no settlmment in the image
"""

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ee
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [ ]:
#Authenticaing API access
ee.Authenticate()

In [ ]:
#Initialzing Earth Engine
ee.Initialize(project='sudan-1234')

In [ ]:
#loading in the LANDSAT7 image data
COPERNICUS = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

In [ ]:
#loading in village data
village_data = pd.read_csv('../content/drive/My Drive/Sudan_village_data/village_data.csv')

In [ ]:
village_data.head()

In [ ]:
#add id column
village_data['ID'] = village_data.index

In [ ]:
#dropping irrelevant columns
village_data = village_data[["ID","LAT_DD", "LONG_DD", "STATUS", "YR_CONFIRM"]]

In [ ]:
village_data = village_data[:1000]

In [ ]:
#spliting yr_confirm into a list of multiple years
village_data['YR_CONFIRM'] = village_data['YR_CONFIRM'].str.split('/')

#selecting the last year
village_data['YR_CONFIRM'] = village_data['YR_CONFIRM'].apply(lambda x: x if x is np.nan else x[-1])


In [ ]:
#plotting histograms of categorial data
village_data['STATUS'].hist()
plt.show()

village_data['YR_CONFIRM'] = village_data['YR_CONFIRM'].astype(float)
village_data['YR_CONFIRM'].hist()
plt.show()

In [ ]:
#value counts for YR_CONFIRM
village_data['YR_CONFIRM'].value_counts()

In [ ]:
village_data["STATUS"].value_counts()

In [ ]:
#seperating the village data into one dataframe of of damaged and destroyed villages and one of undamaged villages
damaged_villages = village_data[village_data['STATUS'].isin(["DAMAGED", "DESTROYED"])]
undamaged_villages = village_data[village_data['STATUS'] == "NO DAMAGE"]

In [ ]:
#merging status of damaged villages into one category called "DAMAGED"
damaged_villages['STATUS'] = damaged_villages['STATUS'].apply(lambda x: 'DAMAGED')

In [ ]:
#limiting damaged villages to be after 2006
damaged_villages = damaged_villages[damaged_villages['YR_CONFIRM'] >= 2006]


In [ ]:
damaged_villages["STATUS"].hist()

In [ ]:
print(len(damaged_villages))
print(len(undamaged_villages))

In [ ]:
#speicifying year 2017
start_date = '2017-04-01'
end_date = '2017-12-31'

COPERNICUS = COPERNICUS.filterDate(start_date, end_date)

In [ ]:
#creates a feature collection of all the village data from LANDSAT7 data, geometry box with 50 meter buffer around coordinates for village and two properties: village status (No damage, or damage) and id

no_damage = ee.FeatureCollection(undamaged_villages.apply(lambda x: ee.Feature(ee.Geometry.BBox(x['LONG_DD']-0.0005, x['LAT_DD']-0.0005, x['LONG_DD']+0.0005, x['LAT_DD']+0.0005), {'STATUS': 'NO DAMAGE', 'ID': x['ID']}), axis=1).tolist())
damaged = ee.FeatureCollection(damaged_villages.apply(lambda x: ee.Feature(ee.Geometry.BBox(x['LONG_DD']-0.0005, x['LAT_DD']-0.0005, x['LONG_DD']+0.0005, x['LAT_DD']+0.0005), {'STATUS': 'DAMAGED', 'ID': x['ID']}), axis=1).tolist())



In [ ]:
#combine into single feature collection
villages = ee.FeatureCollection([no_damage, damaged]).flatten()

In [ ]:
villages.first().getInfo()

In [ ]:
#make a cloud free composite for sentinel copernicus data and rename bands to those of first entry in copernicus
composite = COPERNICUS.reduce(ee.Reducer.median()).rename(COPERNICUS.first().bandNames())

In [ ]:
type(composite)

In [ ]:
#specify bands and property name also add B8 - B12
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B5', 'B6', 'B7']


In [ ]:
#create training data by clipping image at geometry



def get_image(feature):
  clip = composite.select(bands).clip(feature.geometry())
  return clip.set('ID', feature.get('ID')).set('STATUS', feature.get('STATUS'))



training = villages.map(get_image).toList(villages.size())
training = ee.ImageCollection(training)




In [ ]:
type(training)

In [ ]:
#add NBR to training making sure NBR is float64
def add_nbr(image):
  nbr = image.normalizedDifference(['B8', 'B11']).rename('NBR').toDouble()
  return image.addBands(nbr)

training = training.map(add_nbr)



In [ ]:
training.first().getInfo()

In [243]:
#map over image collection
def getStatus(image):
  return ee.Feature(None, {'STATUS' : image.get('STATUS')})

statuses = training.map(getStatus)

statuses.first().getInfo()

#get all statuses from  statuses, convert to list, export as csv
statuses = statuses.toList(statuses.size())
statuses = [ee.Feature(statuses.get(i)).get('STATUS').getInfo() for i in range(statuses.size())]
statuses = pd.DataFrame(statuses)
statuses.to_csv('statuses.csv')


KeyboardInterrupt: 

In [ ]:

#export image collection
def exportImage(image):
  id = image.get('ID').getInfo()
  task = ee.batch.Export.image.toDrive(
      image = image,
      description = 'village_image',
      folder = 'village_images',
      fileNamePrefix = id,
      scale = 10,
      region = image.geometry(),
      maxPixels = 1e13
  )
  task.start()


training = training.toList(training.size())
for i in range(training.size().getInfo()):
  exportImage(ee.Image(training.get(i)))


In [ ]:
#print length of training
print(training.size().getInfo())

In [ ]:
#count number of files in village images folder
path = '../content/drive/MyDrive/village_images'
files = os.listdir(path)
print(len(files))